# Lab03: Web Crawler (Continue) & Information Retrieval.

- MSSV: 18120061
- Họ và tên: Lê Nhựt Nam

## Yêu cầu bài tập

**Cách làm bài**


Bạn sẽ làm trực tiếp trên file notebook này; từ `TODO` cho biết những phần mà bạn cần phải làm.

Bạn có thể thảo luận ý tưởng cũng như tham khảo các tài liệu, nhưng *code và bài làm phải là của bạn*. 

Nếu vi phạm thì sẽ bị 0 điểm cho bài tập này.

**Cách nộp bài**

Trước khi nộp bài, rerun lại notebook (`Kernel` -> `Restart & Run All`).

Sau đó, tạo thư mục có tên `MSSV` của bạn (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`) Chép file notebook, file `t_data.txt` và file `raw_data` của các bạn (nếu file này kích thước lớn các bạn có thể chép link vào `link_data.txt`), nén thư mục `MSSV` này lại và nộp trên moodle.

**Nội dung bài tập**

Cài đặt một web crawler để thu thập dữ liệu từ: https://en.wikipedia.org/wiki/Web_mining.

## Nội dung bài tập

Cài đặt một Web crawler đơn giản bắt đầu từ URL: https://en.wikipedia.org/wiki/Web_mining, tìm liên kết và thu thập dữ liệu trong HTML tại URL này sau đó lặp lại với các URL vừa tìm được.


In [1]:
import requests
import re
from bs4 import BeautifulSoup
from bs4.element import Comment
import string
import pickle

## 1. Thu thập đường dẫn

- Robot.txt:  https://en.wikipedia.org/robots.txt
- **Bước 1**: Thu thập đường dẫn từ https://en.wikipedia.org/wiki/Web_mining. Lưu trữ vào một danh sách `url_list`. 
- **Bước 2**: Lặp lại bước 1 cho các đường dẫn trong `url_list` (**lưu ý:** kiểm tra các đường dẫn vừa thu được đã nằm trong `url_list` hay không?). Dừng khi đã thu thập được 200 URLs.

In [2]:
import random 
import urllib.robotparser

UAS = ("Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1", 
       "Mozilla/5.0 (Windows NT 6.3; rv:36.0) Gecko/20100101 Firefox/36.0",
       "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10; rv:33.0) Gecko/20100101 Firefox/33.0",
       "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36",
       "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36",
       "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36",
       )

ua = UAS[random.randrange(len(UAS))]

rp = urllib.robotparser.RobotFileParser()
rp.set_url("https://en.wikipedia.org/robots.txt")
rp.read()
rrate = rp.request_rate("*")
rp.crawl_delay("*")

head = {
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
  'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
  'Accept-Encoding': 'none',
  'Accept-Language': 'en-US,en;q=0.8',
  'Connection': 'keep-alive',
  'refere': 'https://example.com',
  'cookie': """your cookie value ( you can get that from your web page) """
}

def get_urls(url):
    try:
      r = requests.get(url, headers=head, stream=True, timeout=5)
    except requests.exceptions.ConnectionError as err:
      print(err)
      return None
    # TODO
    # Lấy các url nằm trong trang web của url này, lưu lại vào biến urls
    findall_urls = lambda r : re.findall(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", r.text)
    all_urls = findall_urls(r)
    cp_all_urls = all_urls.copy()
    for url in all_urls:
      if (not rp.can_fetch("*", url)):
          cp_all_urls.remove(url)
    return cp_all_urls

def get_urls_recursive(start_url, limit):
    urls = [start_url]
    for url in urls:
        # TODO
        # Lấy các url nằm trong trang web của url này, lưu lại vào biến new_urls
        new_urls = get_urls(url)
        if new_urls is None: 
          return None
        # Với mỗi url mới trong new_urls:
        for new_url in new_urls:
        #   Nếu nó chưa nằm trong urls thì thêm nó vô  
            if new_url not in urls:
                urls.append(new_url)
        # Nếu kích thước của urls vượt quá limit thì dừng và xóa phần dư thừa
        if len(urls) >= limit:
            urls=urls[0:limit]
            break
    return urls
url_list = get_urls_recursive('https://en.wikipedia.org/wiki/Web_mining', 20)
if url_list is not None:
  url_list_cp = url_list.copy()
  print('\n'.join(map(str, enumerate(url_list, start=1))))

(1, 'https://en.wikipedia.org/wiki/Web_mining')
(2, 'https://creativecommons.org/licenses/by-sa/3.0/')
(3, 'https://archive.org/details/webinformationsy00ngua')
(4, 'https://archive.org/details/webinformationsy00ngua/page/n33')
(5, 'https://archive.org/details/electroniccommer00bauk_698')
(6, 'https://archive.org/details/electroniccommer00bauk_698/page/n176')
(7, 'https://archive.org/details/webminingapplica0000scim/page/282')
(8, 'http://alexandria.tue.nl/repository/freearticles/612259.pdf')
(9, 'http://www.cis.unisa.edu.au/')
(10, 'https://ieeexplore.ieee.org/document/8765250')
(11, 'https://doi.org/10.1109%2FICWR.2019.8765250')
(12, 'https://api.semanticscholar.org/CorpusID:198146435')
(13, 'https://docs.google.com/open?id=1nU1vrz-gBtSJk3bkb1ls_QuGX2nUPPemECPFlCx0C75MvmQdSqPci6LZWJYf')
(14, 'https://ui.adsabs.harvard.edu/abs/2000cs.......11033K')
(15, 'https://doi.org/10.1145%2F360402.360406')
(16, 'https://api.semanticscholar.org/CorpusID:60455')
(17, 'http://library.ifla.org/148/'

## 2. Thu thập dữ liệu
Thu thập dữ liệu từ `url_list`. Lưu trữ dữ liệu thu được vào dictionary data với keys là các từ, values gồm 2 phần tử: 
- `url_idx_list` với $idx \in \left[0,200\right) \cap \mathbb{N}$
- `frequency` 
    
Ví dụ: `data['at']=[url_idx_list,frequency]`:
- `url_idx_list`: danh sách các url mà trong dữ liệu của chúng (html document) chứa từ "at". 
- `frequency`: tần suất xuất hiện (số lần xuất hiện) của từ `at` trong dữ liệu của **tất cả đường dẫn thu được**.   

In [3]:
def text_filter(element):
    # TODO
    # Cài đặt lại như Lab02
    if element.parent.name in ['style', 'title', 'script', 'head', '[document]', 'class', 'a', 'li']:
        return False
    elif isinstance(element, Comment):
        '''Opinion mining?'''
        return False
    elif re.match(r"[\s\r\n]+",str(element)): 
        '''space, return, endline'''
        return False
    return True

def wordList(url):
    # TODO
    # Cài đặt lại như Lab02
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser", from_encoding="UTF-8")
    text = soup.findAll(text=True)
    filtered_text = list(filter(text_filter, text)) # list của các chuỗi
    word_list = []
    # TODO:
    # Với mỗi chuỗi trong filtered_text:
    for text in filtered_text:
        #   Thay thế các dấu câu thành khoảng trắng (gợi ý: danh sách các dấu câu: string.punctuation; thay thế: .replace(...))
        #   Sử dụng .join() function, nếu một char c nào đó nằm trong string.punctuation thì nó sẽ được thay thế bằng ' '. Ngược lại thì không thay đổi gì
        # sau đó join với một string rỗng để có được clean_text
        clean_text = ''.join(' ' if c in string.punctuation else c for c in text)
        #   Tách chuỗi bởi khoảng trắng (.split(...))
        # cắt clean_text vừa có ở trên ra
        lst_words_in_clean_text = clean_text.split()
        #   Thêm các từ vừa được tách ra vào word_list
        word_list = word_list + lst_words_in_clean_text
    return word_list

def read_url(url, url_idx, data):
    # TODO
    # Cài đặt lại như Lab02
    word_list = wordList(url)
    # TODO
    # Với mỗi từ w trong word_list:
    for i, word in enumerate(word_list):
        #   Nếu w chưa có trong data thì khởi tạo data[w] = [[url_idx], 1]
        if word not in data.keys():
            data[word]=[[url_idx],1]
        #   Ngược lại thì thêm url_idx vào data[w][0] (nếu chưa có) và tăng data[w][1] lên 1 đơn vị
        else:
            if url_idx in data[word][0]:
                data[word][0].append(i)
            data[word][1]+=1

In [4]:
data = {}
for url_index, url in enumerate(url_list, 1):
    print(url)
    try:
      read_url(url, url_index, data)
    except:
      pass

https://en.wikipedia.org/wiki/Web_mining
https://creativecommons.org/licenses/by-sa/3.0/
https://archive.org/details/webinformationsy00ngua
https://archive.org/details/webinformationsy00ngua/page/n33
https://archive.org/details/electroniccommer00bauk_698
https://archive.org/details/electroniccommer00bauk_698/page/n176
https://archive.org/details/webminingapplica0000scim/page/282
http://alexandria.tue.nl/repository/freearticles/612259.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


http://www.cis.unisa.edu.au/
https://ieeexplore.ieee.org/document/8765250
https://doi.org/10.1109%2FICWR.2019.8765250
https://api.semanticscholar.org/CorpusID:198146435
https://docs.google.com/open?id=1nU1vrz-gBtSJk3bkb1ls_QuGX2nUPPemECPFlCx0C75MvmQdSqPci6LZWJYf
https://ui.adsabs.harvard.edu/abs/2000cs.......11033K
https://doi.org/10.1145%2F360402.360406
https://api.semanticscholar.org/CorpusID:60455
http://library.ifla.org/148/
https://doi.org/10.5195%2Fjmla.2019.650
http://soave.isti.cnr.it/%7Esilvestr/wp-content/uploads/2007/03/p63-baraglia.pdf
http://facweb.cs.depaul.edu/mobasher/classes/ect584/papers/cms-kais.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


## 3. Tiền xử lý
Loại bỏ các item trong data mà key là các stopword.

**Ngữ liệu:** 

In [5]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
english_stopwords = stopwords.words('english')
print(english_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'ea

In [6]:
# TODO
# Loại bỏ các key của biến data mà nằm trong danh sách english_stopwords
data_copy = data.copy()
for key in data_copy:
  if key in english_stopwords:
    data.pop(key)

## 4. Lưu trữ và biểu diễn dữ liệu
Sử dụng pickle lưu lại data với tên file raw_data.
### 4.1 Cơ sở dữ liệu giao tác:
Thông thường, các cơ sở dữ liệu giao tác được lưu trong flat files (các tập phẳng) thay vì trong một hệ cơ sở dữ liệu. Các item là các số nguyên không âm, mỗi giao tác tương ứng với một dòng các số nguyên phân tách nhau bằng khoảng trắng.
Ví dụ:

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

30 31 32 

33 34 35 

36 37 38 39 40 41 42 43 44 45 46 

38 39 47 48 

38 39 48 49 50 51 52 53 54 55 56 57 58 

32 41 59 60 61 62 

3 39 48 

63 64 65 66 67 68 

32 69 

48 70 71 72 

39 73 74 75 76 77 78 79 

36 38 39 41 48 79 80 81 

82 83 84 

41 85 86 87 88 

39 48 89 90 91 92 93 94 95 96 97 98 99 100 101 

36 38 39 48 89 

39 41 102 103 104 105 106 107 108 

38 39 41 109 110 

39 111 112 113 114 115 116 117 118 

119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 

48 134 135 136 

39 48 137 138 139 140 141 142 143 144 145 146 147 148 149 

39 150 151 152 

38 39 56 153 154 155 

In [7]:
with open('raw_data', 'wb') as f:
    # TODO
    pickle.dump(data,f)

In [8]:
output = {}
with open("raw_data", "rb") as f:
    output = pickle.load(f)

from itertools import islice
list(islice(output.items(), 10))

[('Web',
  [[1,
    56,
    88,
    92,
    102,
    106,
    111,
    133,
    136,
    146,
    149,
    152,
    162,
    170,
    173,
    176,
    192,
    195,
    210,
    219,
    221,
    237,
    251,
    326,
    520,
    541,
    545,
    560,
    563,
    579,
    639,
    701,
    718,
    774,
    795,
    801,
    818,
    844,
    886,
    903],
   139]),
 ('mining',
  [[1,
    57,
    85,
    89,
    94,
    104,
    108,
    113,
    134,
    137,
    148,
    151,
    154,
    163,
    168,
    172,
    175,
    178,
    194,
    197,
    223,
    253,
    293,
    311,
    328,
    461,
    495,
    522,
    543,
    547,
    557,
    562,
    565,
    568,
    629,
    641,
    702,
    720,
    747],
   116]),
 ('From', [[1], 4]),
 ('Wikipedia', [[1, 971], 3]),
 ('free', [[1], 7]),
 ('encyclopedia', [[1], 1]),
 ('‹', [[1], 1]),
 ('The', [[1, 36, 40, 155, 289, 397, 439, 582, 684, 725], 50]),
 ('considered', [[1, 236, 343, 454], 4]),
 ('merging', [[1], 1])]

### 4.2 Xuất dataset
Lưu một cơ sở dữ liệu giao tác (transactional database) vào file t_data.txt: 
- Các item tương ứng với url_idx
- Mỗi transaction tương ứng với một từ.

In [9]:
with open('t_data.txt', 'w') as f:
    for word, (url_list, freq) in data.items():
        print(*url_list, file=f)

## 5. Truy vấn and, or, not
Ví dụ: 
- Truy vấn `and` câu `web mining`: trả về đường dẫn tới các trang web có cả 2 từ web và từ mining. 
- Truy vấn `or` câu `web mining`: trả về đường dẫn tới các trang web có từ web hoặc từ mining.
- Truy vấn `not` câu `web mining`: trả về đường dẫn tới các trang không có cả từ web và từ mining.

*GỢI Ý: TÁCH CÂU TRUY VẤN THÀNH CÁC TỪ TƯƠNG TỰ PHƯƠNG PHÁP LÀM Ở LAB02.*

In [10]:
url_list

[20, 55534]

In [24]:
ret=[]
def andRetrieval(ret, sentence):
    '''Parameters
    -----------------------
    ret: url_list
    sentence: query'''
    # TODO
    ### split sentence (separator ' ') into terms
    terms = sentence.split(" ")
    ### find urls have all terms: urls
    flag = 0
    for url in url_list_cp:
      for term in terms:
        if (url.find(term) == -1):
          flag = 0
          break
        else:
          flag = 1
          continue
      if (flag):
        ret.append(url)
    ### if len(ret)==0:  return urls
    if len(ret)==0:
      return ret
    ### else update ret with urls: intersection of ret and urls 
    else:
      return ret


def orRetrieval(ret, sentence):
    '''Parameters
    -----------------------
    ret: url_list
    sentence: query'''
    # TODO
    ### split sentence (separator ' ') into terms
    terms=sentence.split(" ")
    ### find urls have all terms: urls
    for url in url_list_cp:
      for term in terms:
    ### find urls have at least 1 term: urls
        if (url.find(term)):
          ret.append(url)
          break
    ### update ret with urls: extend ret with urls
    return ret

def notRetrieval(ret, sentence):
    '''Parameters
    -----------------------
    ret: url_list
    sentence: query'''
    # TODO
    ### split sentence (separator ' ') into terms
    terms=sentence.split(" ")
    ### find urls have at least 1 term: urls
    urls = []
    flag = 0
    for url in url_list_cp:
      for term in terms:
        if (url.find(term)):
          flag = 0
          break
        else:
          flag = 1
      if flag==1:
        urls.append(url)
    ### update ret with urls: remove urls from ret 
    ret = list(set(ret) - set(urls))
    return ret

In [12]:
print(andRetrieval([], 'web mining'))

['https://archive.org/details/webminingapplica0000scim/page/282']


In [13]:
print(orRetrieval([], 'web mining'))

['https://en.wikipedia.org/wiki/Web_mining', 'https://creativecommons.org/licenses/by-sa/3.0/', 'https://archive.org/details/webinformationsy00ngua', 'https://archive.org/details/webinformationsy00ngua/page/n33', 'https://archive.org/details/electroniccommer00bauk_698', 'https://archive.org/details/electroniccommer00bauk_698/page/n176', 'https://archive.org/details/webminingapplica0000scim/page/282', 'http://alexandria.tue.nl/repository/freearticles/612259.pdf', 'http://www.cis.unisa.edu.au/', 'https://ieeexplore.ieee.org/document/8765250', 'https://doi.org/10.1109%2FICWR.2019.8765250', 'https://api.semanticscholar.org/CorpusID:198146435', 'https://docs.google.com/open?id=1nU1vrz-gBtSJk3bkb1ls_QuGX2nUPPemECPFlCx0C75MvmQdSqPci6LZWJYf', 'https://ui.adsabs.harvard.edu/abs/2000cs.......11033K', 'https://doi.org/10.1145%2F360402.360406', 'https://api.semanticscholar.org/CorpusID:60455', 'http://library.ifla.org/148/', 'https://doi.org/10.5195%2Fjmla.2019.650', 'http://soave.isti.cnr.it/%7Es

In [25]:
print(notRetrieval(url_list_cp, 'web mining'))

['http://facweb.cs.depaul.edu/mobasher/classes/ect584/papers/cms-kais.pdf', 'https://en.wikipedia.org/wiki/Web_mining', 'https://archive.org/details/webminingapplica0000scim/page/282', 'https://archive.org/details/webinformationsy00ngua/page/n33', 'https://archive.org/details/webinformationsy00ngua']
